# Draw Tree Diagram

Last Edit: 3/3/17

This script is designed to generate a tree diagram with nodes that the user inputs. This feature is meant to compliment the visualize flux diagram script by allowing the user to specify the network for use in analyzing complex mechanisms in conjunction with the Mechanism Analyzer script. 

## Import Necessary Modules

In [1]:
import pydot
import os
from IPython.display import Image

from rmgpy.rmg.model import CoreEdgeReactionModel
from rmgpy.chemkin import loadChemkinFile
from rmgpy.chemkin import getSpeciesIdentifier

## User Input: File Paths

In [ ]:
########## INPUT ##########
CurrentWorkingDirectory='/home/mark/RMG/Mark_RMG/DrawTreeDiagram/' # MUST BE THE FULL PATH 
ChemkinFilePath='../chem.inp'
SpeciesDictionaryPath='../species_dictionary.txt'
MoleculesOutputPath='../Molecules/'
GraphDictionaryFile='../GraphDictionary.txt'
graphFile='../FluxDiagram.png'
###########################

## Load Chemkin and Species Dictionary Files

In [ ]:
model = CoreEdgeReactionModel()
model.core.species, model.core.reactions = loadChemkinFile(ChemkinFilePath,SpeciesDictionaryPath)

# Create species_identifier_didct for the core species    
species_identifier_dict = {}
for i, s in enumerate(model.core.species):
    species_identifier_dict.update({getSpeciesIdentifier(model.core.species[i]): s})

## Initialize Graph

In [ ]:
def newgraph():
    graph=pydot.Dot('flux_diagram',graph_type='digraph', overlap='flase')
    #graph.set_rankdir('LR')  # Uncomment if you want graph to flow from left to right
    graph.set_fontname('sans')
    graph.set_fontsize('10')
    return graph
graph=newgraph()
nodeList=[]
myGraphDictionary={}

## Define Necessary Functions

In [ ]:
def loadDictionaryGraph(dictionary):
    newnodelist=[]
    for s in dictionary:
        T=''
        F=''
        Flux=dictionary[s]
        x=0
        for i in s:
            if i=='&':
                F=s[x+1:]
                break
            T=T+i
            x=x+1
        newnodelist=addFlux(T,F,Flux,newnodelist)
        
    dictionaryfile=open(GraphDictionaryFile,'a')
    dictionaryfile.write(str(myGraphDictionary)+'\n'+'\n')
    dictionaryfile.close()
    
    return newnodelist
        
def addFlux(ToSpecies,FromSpecies,Flux,nodeList):
    Units='mol'
    
    if FromSpecies not in nodeList:
        nodeList=nodeList+[FromSpecies]
        PicturePath=os.path.join(CurrentWorkingDirectory,MoleculesOutputPath,FromSpecies+'.png')
        species_identifier_dict[FromSpecies].molecule[0].draw(PicturePath)
        nodeLabel = "<<TABLE border='0'><TR><TD><IMG SRC='{0}'/></TD></TR></TABLE>>".format(PicturePath)
        graph.add_node(pydot.Node(FromSpecies, label=nodeLabel))
                                                              
                                                              
                                                              
    if ToSpecies not in nodeList:
        nodeList=nodeList+[ToSpecies]
        PicturePath=os.path.join(CurrentWorkingDirectory,MoleculesOutputPath,ToSpecies+'.png')
        species_identifier_dict[ToSpecies].molecule[0].draw(PicturePath)
        nodeLabel = "<<TABLE border='0'><TR><TD><IMG SRC='{0}'/></TD></TR></TABLE>>".format(PicturePath)
        graph.add_node(pydot.Node(ToSpecies, label=nodeLabel))
        
    graph.add_edge(pydot.Edge(FromSpecies,ToSpecies, label=Flux+'\n'+Units))
    return nodeList

    
def UpdateGraphDictionary(ToSpecies,FromSpecies,Flux,myGraphDictionary):
    myGraphDictionary.update({ToSpecies+'&'+FromSpecies:Flux})
    dictionaryfile=open(GraphDictionaryFile,'a')
    dictionaryfile.write(str(myGraphDictionary)+'\n'+'\n')
    dictionaryfile.close()
    return myGraphDictionary
        

# Functionality 1: Add Species/Flux to Diagram

To a flux from one species to another on the diagram, enter the name of the species exactly as they appear in the species_dictionary.txt file as a string, then enter the flux in moles (or change the units in the addFlux function).

In [ ]:
####### INPUT #########
FromSpecies='PDD(1)'
ToSpecies='C7(32)'
Flux=str('{:0.3e}'.format(9.459E-8))
#######################

nodeList=addFlux(ToSpecies,FromSpecies,Flux,nodeList)
myGraphDictionary=UpdateGraphDictionary(ToSpecies,FromSpecies,Flux,myGraphDictionary)
graph.write_png(graphFile)
Image(graphFile)


# Functionality 2: Remove a Flux from the Diagram

In [ ]:
########### INPUT #############
ToSpecies='C8(33)'
FromSpecies='PDD(1)'
###############################


del myGraphDictionary[ToSpecies+'&'+FromSpecies]


graph=newgraph()
nodeList=loadDictionaryGraph(myGraphDictionary)
graph.write_png(graphFile)
Image(graphFile)


# Functionality 3: Load Graph from Graph Dictionary

If the graph was made using this script, a graph dictionary is stored in the GraphDictionaryFile specified in file paths. Each time you update the graph it appends the new graph dictionary to this file. To load a graph simply copy the desired graph dictionary into the input below and run.

In [ ]:
########## INPUT ##########
myGraphDictionary={'C8(33)&ETHBENZ(4)': '1.192e-08', 'C8(33)&S(42583)': '1.948e-08', 'OCTYL(54)&C11H23(97)': '3.084e-05', 'C8ene(43)&C8(33)': '9.459e-08', 'C8(33)&S(1253)': '2.927e-08', 'S(1254)&C8(33)': '2.26e-07', 'OCTYL(54)&C8(33)': '3.34e-05'}
###########################

graph=newgraph()
nodeList=loadDictionaryGraph(myGraphDictionary)
graph.write_png(graphFile)
Image(graphFile)